In [1]:
import sys
sys.path.append('..')
import umbrella_mesh
import linkage_vis
import load_jsondata
from configuration import *
from visualization_helper import *

In [2]:
import numpy as np

In [3]:
sys.path.append('../UmbrellaGen')
import grid_gen

### Define the grid dimensions and Umbrella Valence you want

In [4]:
degree = 3 # 3 for TriUmbrellas, 4 for QuadUmbrellas
rows = 5 # please make sure rows >=2
cols = 5 # please make sure cols >=2

In [5]:
numUmbrellas = 0
if degree == 3:
    numUmbrellas = (rows * 2) * cols
elif degree == 4 or degree == 6:
    numUmbrellas = rows * cols

### Define your favorite height, material and cross-section     #-------function! Umbrellas are sequenced row-wise

In [6]:
def myHeightFunc(numUmbrellas): 
    heights = [1] * numUmbrellas # Scale Factor of heights. Set minHeight to your desired minimum absolute height.
    for uid in range(numUmbrellas):
        heights[uid] += (0.1 * uid**1.6 + 0.1 *(numUmbrellas - 1 - uid)**1.6)
        # heights[uid] += (0.01 * uid**3 + 0.01 *(numUmbrellas - 1 - uid)**3)
    print(min(heights))
    heights = [h/min(heights) for h in heights]
    # print(np.array(heights)*64)
    return heights

def myMaterialFunc(numUmbrellas):
    plate_E, plate_nu, plate_thickness, plate_width = [1400 * 10, 0.35, 3.0, 5.0]
    E = [1400] * numUmbrellas #Default values for uniform material umbrellas
    nu = [0.35] * numUmbrellas
    thickness = [3.0] * numUmbrellas
    width = [5.0] * numUmbrellas
    
    # Write your material function here dependent on UmbrellaID (uid). In the visualization below the colormap (viridis) will help you visualize the IDs. They are sequenced row-wise
    #-------
    for uid in range(numUmbrellas):
        # E[uid] += -2000 + 0.05 * (  uid ** 3 + (numUmbrellas - 1 - uid)**3 )
        # nu[uid] += 0.01 * uid
        thickness[uid] += 0.1 * uid
        width[uid] += 0.1 * uid
    #-------
        
        
    material_params = np.zeros((1 + 4 * (numUmbrellas + 1)))
    material_params[0] = numUmbrellas + 1
    material_params[1::4] = E + [plate_E]
    material_params[2::4] = nu + [plate_nu]
    material_params[3::4] = thickness + [plate_thickness]
    material_params[4::4] = width + [plate_width]
    
    return material_params
    

In [7]:

grid_gen.genUmbrellaWithHeights(degree, rows, cols, height_scales = myHeightFunc(numUmbrellas),minHeight = 64, useOverHang = True)
# grid_gen.genUmbrellaWithHeights(degree, rows, |cols, None)

34.40277486269047


### Initialization

In [8]:
name = 'grid_dump'
input_path = '../UmbrellaGen/{}.json.gz'.format(name)
io1, input_data1, target_mesh, curr_um1, plate_thickness, target_height_multiplier = parse_input(input_path, handleBoundary = False, isHex = (degree == 6))


In [9]:
grid_gen.genUmbrellaWithHeights(degree, rows, cols, height_scales = myHeightFunc(numUmbrellas),minHeight = 64, useOverHang = True, armPlateEdgeAxisOffset = 2.5, armJointAxisOffset = 2.5)
io2, input_data2, target_mesh, curr_um2, plate_thickness, target_height_multiplier = parse_input(input_path, handleBoundary = False, isHex = (degree == 6))

34.40277486269047


In [10]:
rod_colors1 = get_color_field(curr_um1, input_data1, uidBased = False) 
rod_colors2 = get_color_field(curr_um2, input_data2, uidBased = False)

from ipywidgets import HBox
lview1 = linkage_vis.LinkageViewer(curr_um1, width=800, height=600)
lview1.update(scalarField = rod_colors1)
lview2 = linkage_vis.LinkageViewer(curr_um2, width=800, height=600)
lview2.update(scalarField = rod_colors2)
HBox([lview1.show(), lview2.show()])



In [11]:
import time
curr_um1.numSegments(),curr_um2.numSegments(), curr_um1.numJoints(),curr_um2.numJoints()

(820, 820, 725, 725)

### Deploy first umbrellaMesh - UmbrellaMesh with uniform material/cross-section everywhere

In [12]:
start = time.time()
success = deploy_umbrella_pin_rigid_motion(curr_um1, plate_thickness, target_height_multiplier, lview1, rod_colors1)
print(success, time.time() - start)

True 10.779706954956055


### Deploy first umbrellaMesh - UmbrellaMesh with varying material/cross-section

In [13]:
start = time.time()
success = deploy_umbrella_pin_rigid_motion(curr_um2, plate_thickness, target_height_multiplier, lview2, rod_colors2)
print(success, time.time() - start)

True 13.840172052383423


### Align Views appropriately

In [14]:
# assert 0
# lview1.setCameraParams(lview2.getCameraParams())
lview2.setCameraParams(lview1.getCameraParams())

In [15]:
V1, F1, _ = curr_um1.visualizationGeometry()
V2, F2, _ = curr_um2.visualizationGeometry()
V3 = np.vstack((V1, V2))
F3 = np.vstack((F1, F2 + len(V1)))

import vis
from matplotlib import cm
bending1, bending2 = curr_um1.maxBendingStresses(), curr_um2.maxBendingStresses()
bvmin, bvmax = np.min(bending1 + bending2), np.max(bending1 + bending2)
print(bvmin, bvmax)
twisting1, twisting2 = curr_um1.twistingStresses(), curr_um2.twistingStresses()
tvmin, tvmax = np.min(twisting1 + twisting2), np.max(twisting1 + twisting2)
print(tvmin, tvmax)

print(np.array(bending1).shape)
print(np.max(bending1), np.max(bending2))
print(np.max(twisting1), np.max(twisting2))


print((np.max(bending2) - np.max(bending1))/np.max(bending1))
print((np.max(twisting2) - np.max(twisting1))/np.max(twisting1))


pos1 = np.array([curr_um1.segment(sid).rod.deformedPoints() for sid in range(curr_um1.numSegments())])
pos2 = np.array([curr_um2.segment(sid).rod.deformedPoints() for sid in range(curr_um2.numSegments())])
distance = np.linalg.norm(pos2 - pos1, axis = -1)


print(distance.shape, len(bending1), curr_um1.numSegments(), curr_um1.numJoints())

sf1b = vis.fields.ScalarField(curr_um1, bending1, colormap = cm.plasma, vmin = bvmin, vmax = bvmax)
sf2b = vis.fields.ScalarField(curr_um2, bending2, colormap = cm.plasma, vmin = bvmin, vmax = bvmax)
sf1t = vis.fields.ScalarField(curr_um1, twisting1, colormap = cm.plasma, vmin = tvmin, vmax = tvmax)
sf2t = vis.fields.ScalarField(curr_um2, twisting2, colormap = cm.plasma, vmin = tvmin, vmax = tvmax)

sfd = vis.fields.ScalarField(curr_um2, distance, colormap = cm.plasma)


color1b, color2b = sf1b.colors(), sf2b.colors()
color1t, color2t = sf1t.colors(), sf2t.colors()
color1d, color2d = np.ones_like(sfd.colors())*0.5, sfd.colors()

bcolors = np.vstack((color1b, color2b))
tcolors = np.vstack((color1t, color2t))
dcolors = np.vstack((color1d, color2d))
bview, tview, dview = linkage_vis.LinkageViewer((V3, F3)), linkage_vis.LinkageViewer((V3, F3)), linkage_vis.LinkageViewer((V3, F3))
bview.update(scalarField = bcolors)
tview.update(scalarField = tcolors)
dview.update(scalarField = dcolors)
HBox([bview.show(), tview.show(), dview.show()])

0.0 20.656874965536527
0.0 5.96995477550483
(820, 11)
18.212748008396563 20.656874965536527
4.421484255030104 5.96995477550483
0.13419869181812413
0.3502150932038505
(820, 11) 820 820 725


In [16]:
tview.setCameraParams(bview.getCameraParams())
dview.setCameraParams(bview.getCameraParams())
assert 0

AssertionError: 

### Max vonMises Stress Visualization

In [ ]:
lview1.update(scalarField = curr_um1.maxVonMisesStresses())
lview2.update(scalarField = curr_um2.maxVonMisesStresses())

### Release actuation forces to see if there is a bistable state

In [ ]:
deploy_umbrella_pin_rigid_motion(curr_um1, plate_thickness, target_height_multiplier, lview1, rod_colors1, releaseActuation = True)

In [ ]:
deploy_umbrella_pin_rigid_motion(curr_um2, plate_thickness, target_height_multiplier, lview2, rod_colors2, releaseActuation = True)